# SET UP

## Import libraries and data

Data in RAW_DATA_DIR path

In [ ]:
%run ../paths.py
print("RAW_DATA_DIR:", RAW_DATA_DIR)

RAW_DATA_DIR: /Users/ricardoheredia/Desktop/retail-forecasting/data/raw


In [2]:
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import inspect

In [3]:
db_filename = 'supermarket.db'  # Change as needed
database_path = RAW_DATA_DIR / db_filename

In [4]:
con = sa.create_engine('sqlite:///' + str(database_path))

In [5]:
insp = inspect(con)
tables = insp.get_table_names()
tables

['calendar', 'sales', 'sell_prices']

## Create datasets

In [6]:
cal = pd.read_sql('calendar', con)
cal.drop(columns='index',inplace=True)
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2
0,2013-01-01,11249,Tuesday,4,1,2013,d_704,NewYear,National,None,None
1,2013-01-02,11249,Wednesday,5,1,2013,d_705,None,None,None,None
2,2013-01-03,11249,Thursday,6,1,2013,d_706,None,None,None,None
3,2013-01-04,11249,Friday,7,1,2013,d_707,None,None,None,None
4,2013-01-05,11250,Saturday,1,1,2013,d_708,None,None,None,None


In [7]:
sales = pd.read_sql('sales', con)
sales.drop(columns='index',inplace=True)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_704,d_705,d_706,d_707,...,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,0,224,241,232,...,5,2,0,0,6,0,6,0,0,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,54,63,44,0,65,90,104,73,76,97
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,20,23,23,25,...,43,40,39,0,29,33,27,13,26,47
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,34,27,40,49,...,31,43,32,0,52,37,32,29,34,27
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,29,45,28,0,46,36,40,31,46,36


In [8]:
prices = pd.read_sql('sell_prices', con)
prices.drop(columns='index',inplace=True)
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


## Join data

### Understand tables relationships

In [9]:
from IPython import display
display.Image("../../99_Media/CasoRetail_EstructuraRelaciones.jpg")

FileNotFoundError: No such file or directory: '../../99_Media/CasoRetail_EstructuraRelaciones.jpg'

FileNotFoundError: No such file or directory: '../../99_Media/CasoRetail_EstructuraRelaciones.jpg'

<IPython.core.display.Image object>

- Calendar Table:

Contains date-related information
Has columns for date, wm_yr_wk (Walmart year week), weekday, month, year
Important column 'd' (like d_704) which is used as a reference in the sales table
Contains event information (event_name_1, event_type_1, etc.)

- Sales Table:

Contains sales data for different items across stores
Has item_id, store_id, department (dept_id), category (cat_id), and state information
The columns d_704 through d_1798 represent daily sales quantities
These 'd_' columns correspond to the 'd' column in the calendar table
Prices Table (sell_prices):
Contains pricing information for items
Has store_id, item_id, wm_yr_wk, and sell_price
Shows the price of items by week at different stores
The relationships (as shown in the diagram):

- Calendar and Sales tables have a many-to-many relationship (∞)
- Sales and Prices tables are connected through [wm_yr_wk, store_id, item_id] as key fields

What I can do:
- Link sales data to specific dates using the 'd' columns
- Find the price of any item at any store during a specific week
- Analyze sales patterns along with pricing and calendar events

For example, if you want to analyze the sales of FOODS_3_090 at store CA_3:

Use the calendar table to get the dates
Look up the sales quantities from the sales table
Match the pricing from the prices table using wm_yr_wk, store_id, and item_id

PS:

* First join calendar + sales, because to join with prices we need columns from boths.
* Before joining cal and sales, let's modify 'd' to pass it as rows instead of columns.

### Join cal + sales

#### Transform sales

In [10]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_704,d_705,d_706,d_707,...,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,0,224,241,232,...,5,2,0,0,6,0,6,0,0,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,54,63,44,0,65,90,104,73,76,97
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,20,23,23,25,...,43,40,39,0,29,33,27,13,26,47
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,34,27,40,49,...,31,43,32,0,52,37,32,29,34,27
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,29,45,28,0,46,36,40,31,46,36


In [11]:
sales = sales.melt(id_vars = sales.columns[0:6],
                   var_name = 'd',
                   value_name = 'sales')
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0
...,...,...,...,...,...,...,...,...
21895,FOODS_3_329_CA_4_validation,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6
21896,FOODS_3_555_CA_4_validation,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5
21897,FOODS_3_586_CA_4_validation,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11
21898,FOODS_3_587_CA_4_validation,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16


Delete id variable

In [12]:
sales.drop(columns='id',inplace=True)

#### Merge with calendar to include that information

In [13]:
df = sales.merge(right = cal, how = 'left', on = 'd')
df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None


### Add prices table information

In [14]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


In [15]:
df = df.merge(right = prices, how = 'left', on = ['store_id','item_id','wm_yr_wk'])
df

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.98
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.28
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.48
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21895,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,2.98
21896,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,1.68
21897,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,1.68
21898,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,2.48


Let's check

In [16]:
df[['store_id','item_id','wm_yr_wk','d','sell_price']]\
    .sort_values(by = ['store_id','item_id','wm_yr_wk','d'])\
    .head(10)

,store_id,item_id,wm_yr_wk,d,sell_price
0,CA_3,FOODS_3_090,11249,d_704,1.25
20,CA_3,FOODS_3_090,11249,d_705,1.25
40,CA_3,FOODS_3_090,11249,d_706,1.25
60,CA_3,FOODS_3_090,11249,d_707,1.25
80,CA_3,FOODS_3_090,11250,d_708,1.25
100,CA_3,FOODS_3_090,11250,d_709,1.25
120,CA_3,FOODS_3_090,11250,d_710,1.25
140,CA_3,FOODS_3_090,11250,d_711,1.25
160,CA_3,FOODS_3_090,11250,d_712,1.25
180,CA_3,FOODS_3_090,11250,d_713,1.25


### Reorder and set date as index

In [17]:
df.head(1)

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25


In [18]:
order = ['date',
         'state_id',
         'store_id',
         'dept_id',
         'cat_id',
         'item_id',
         'wm_yr_wk',
         'd',
         'sales',
         'sell_price',
         'year',
         'month',
         'wday',
         'weekday',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2']

In [19]:
df = df[order].set_index('date')
df

,state_id,store_id,dept_id,cat_id,item_id,wm_yr_wk,d,sales,sell_price,year,month,wday,weekday,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,,,,,,,,,,,,,
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_090,11249,d_704,0,1.25,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_120,11249,d_704,0,4.98,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_202,11249,d_704,20,4.28,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_252,11249,d_704,34,1.48,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_288,11249,d_704,0,NaN,2013,1,4,Tuesday,NewYear,National,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_329,11548,d_1798,6,2.98,2015,12,6,Thursday,None,None,None,None
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_555,11548,d_1798,5,1.68,2015,12,6,Thursday,None,None,None,None
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_586,11548,d_1798,11,1.68,2015,12,6,Thursday,None,None,None,None


### Save validation and training datasets

Since it's forecasting and we're using dates we can't do it randomly. It needs to contain the last available data. 

In [24]:
val = df['2015-12-01':'2015-12-31']

work = df[:'2015-11-31']

In [25]:
validation_file = 'validation.parquet'

save_path = VALIDATION_DIR / validation_file

val.to_parquet(save_path)

In [26]:
val

,state_id,store_id,dept_id,cat_id,item_id,wm_yr_wk,d,sales,sell_price,year,month,wday,weekday,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,,,,,,,,,,,,,
2015-12-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_090,11544,d_1768,57,1.50,2015,12,4,Tuesday,None,None,None,None
2015-12-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_120,11544,d_1768,109,4.98,2015,12,4,Tuesday,None,None,None,None
2015-12-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_202,11544,d_1768,17,4.58,2015,12,4,Tuesday,None,None,None,None
2015-12-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_252,11544,d_1768,29,1.58,2015,12,4,Tuesday,None,None,None,None
2015-12-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_288,11544,d_1768,25,1.50,2015,12,4,Tuesday,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_329,11548,d_1798,6,2.98,2015,12,6,Thursday,None,None,None,None
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_555,11548,d_1798,5,1.68,2015,12,6,Thursday,None,None,None,None
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_586,11548,d_1798,11,1.68,2015,12,6,Thursday,None,None,None,None


### Extract & Load training dataset

In [27]:
work_file = 'work.parquet'

save_path_work = TRANSFORMED_DATA_DIR / work_file

work.to_parquet(save_path_work)

In [28]:
work 

,state_id,store_id,dept_id,cat_id,item_id,wm_yr_wk,d,sales,sell_price,year,month,wday,weekday,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,,,,,,,,,,,,,
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_090,11249,d_704,0,1.25,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_120,11249,d_704,0,4.98,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_202,11249,d_704,20,4.28,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_252,11249,d_704,34,1.48,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_288,11249,d_704,0,NaN,2013,1,4,Tuesday,NewYear,National,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,CA,CA_4,FOODS_3,FOODS,FOODS_3_329,11544,d_1767,5,2.98,2015,11,3,Monday,None,None,None,None
2015-11-30,CA,CA_4,FOODS_3,FOODS,FOODS_3_555,11544,d_1767,4,1.68,2015,11,3,Monday,None,None,None,None
2015-11-30,CA,CA_4,FOODS_3,FOODS,FOODS_3_586,11544,d_1767,9,1.68,2015,11,3,Monday,None,None,None,None


### No samples for this project.